In [3]:
import re
from bs4 import BeautifulSoup 

In [4]:
def review_to_wordlist(review):
    '''
    Meant for converting each of the IMDB reviews into a list of words.
    '''
    # First remove the HTML.
    review_text = BeautifulSoup(review).get_text()
    
    # Use regular expressions to only include words.
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # Convert words to lower case and split them into separate words.
    words = review_text.lower().split()
   
    # Return a list of words
    return(words)

In [5]:
import pandas as pd

In [17]:
# Import both the training and test data.
train = pd.read_csv('Individual assaignment data mining/Gegeven dataset/labeledTrainData.tsv', header=0,
                delimiter="\t", quoting=3)
test = pd.read_csv('Individual assaignment data mining/Gegeven dataset/testData.tsv', header=0, delimiter="\t",
               quoting=3 )

In [18]:
y_train = train['sentiment']
X_train = train['review']

In [19]:
traindata = []
for i in range(0,len(X_train)):
    traindata.append(" ".join(review_to_wordlist(X_train[i])))
testdata = []
for i in range(0,len(X_train)):
    testdata.append(" ".join(review_to_wordlist(X_train[i])))

/Users/gladwellacademy/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/gladwellacademy/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
'''
ngram_range = (1 ,2)

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
        strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
        ngram_range=ngram_range, use_idf=1,smooth_idf=1,sublinear_tf=1,
        stop_words = 'english')

Xfit = tfv.fit(traindata) 
X = tfv.transform(traindata)

Xtestfit = tfv.fit(testdata)
X_test = tfv.transform(testdata)
'''

"\nngram_range = (1 ,2)\n\ntfv = TfidfVectorizer(min_df=3,  max_features=None, \n        strip_accents='unicode', analyzer='word',token_pattern=r'\\w{1,}',\n        ngram_range=ngram_range, use_idf=1,smooth_idf=1,sublinear_tf=1,\n        stop_words = 'english')\n\nXfit = tfv.fit(traindata) \nX = tfv.transform(traindata)\n\nXtestfit = tfv.fit(testdata)\nX_test = tfv.transform(testdata)\n"

Making a function so it can be pipelined

In [14]:
def tfidf(X_train = None, X_test = None, ngram_range = (1 ,2)):
    from sklearn.feature_extraction.text import TfidfVectorizer
    if ngram_range:
        ngram_range = ngram_range
    else:
        ngram_range = (1 ,2)

    tfv = TfidfVectorizer(min_df=3,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=ngram_range, use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

    if X_train:
        Xfit = tfv.fit(X_train) 
        train = tfv.transform(X_train)
    else: train = 'no training data provided'
    if X_test:
        Xtestfit = tfv.fit(X_test)
        test = tfv.transform(X_test)
    else: test = 'no testing data provided'
    return(train, test)

In [22]:
X_train, X_test = tfidf(X_train = traindata, X_test = testdata)

In [23]:
from sklearn.metrics import classification_report

In [24]:
from sklearn.model_selection import train_test_split

#Splitting labeled dataset in train and test sets to evaluate later on.
rvw_train, rvw_test, label_train, label_test = \
train_test_split(X_train, train['sentiment'], test_size=0.2)

In [25]:
from sklearn.model_selection import train_test_split

#Splitting labeled dataset in train and test sets to evaluate later on.
rvw_train, rvw_test, label_train, label_test = \
train_test_split(X, train['sentiment'], test_size=0.2)

NameError: name 'X' is not defined

###  Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

/Users/gladwellacademy/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/gladwellacademy/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [27]:
grid_values = {'C':[1, 10 ,100]} # Decide which settings you want for the grid search. 


param_grid = [{'dual': [False], 'penalty': ['l1'], 'C': [1, 10]}#, 
              #{'dual': [True], 'penalty':['l2'], 'C': [1, 10 ,100]}]
             ]
model_LR = GridSearchCV(LogisticRegression(penalty = 'L2', dual = True, random_state = 0), 
                        param_grid, scoring = 'roc_auc', cv = 20) 
                        
model_LR.fit(rvw_train,label_train) # Fit the model.

GridSearchCV(cv=20, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='L2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'dual': [False], 'penalty': ['l1'], 'C': [1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [28]:
from sklearn.metrics import classification_report
predictionsLR = model_LR.predict(rvw_test)
print (classification_report(predictionsLR, label_test))

             precision    recall  f1-score   support

          0       0.87      0.89      0.88      2433
          1       0.89      0.88      0.89      2567

avg / total       0.88      0.88      0.88      5000



In [29]:
model_LR.best_estimator_

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Multinimal Naive Bayes

In [30]:
from sklearn.naive_bayes import MultinomialNB

In [31]:
model_MNB = MultinomialNB()

#Fitting model
model_MNB.fit(rvw_train,label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
predictionsMNB = model_MNB.predict(rvw_test)
print (classification_report(predictionsMNB, label_test))

             precision    recall  f1-score   support

          0       0.89      0.87      0.88      2524
          1       0.87      0.89      0.88      2476

avg / total       0.88      0.88      0.88      5000



###  Stochastic Gradient Descent

In [35]:
from sklearn.linear_model import SGDClassifier

In [36]:
#Setting the alpha for a regularization parameter
sgd_params = {'alpha': [0.00006, 0.00007, 0.00008, 0.0001, 0.0005]} 

#Run GridSearch and fitting model 
model_SGD = GridSearchCV(SGDClassifier(random_state = 0, shuffle = True, loss = 'modified_huber'), 
                        sgd_params, scoring = 'roc_auc', cv = 20)

model_SGD.fit(rvw_train,label_train) 

GridSearchCV(cv=20, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [6e-05, 7e-05, 8e-05, 0.0001, 0.0005]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [37]:
predictionsSGD = model_SGD.predict(rvw_test)
print (classification_report(predictionsSGD, label_test))

             precision    recall  f1-score   support

          0       0.89      0.92      0.90      2421
          1       0.92      0.90      0.91      2579

avg / total       0.91      0.91      0.91      5000



In [39]:
model_SGD.best_estimator_

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

### Random Forrest 

In [40]:
from sklearn.ensemble import RandomForestClassifier

#use a large GridSearch over all parameters
RFC param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

#Run GridSearch and fitting model
model_RFC = GridSearchCV(RandomForestClassifier(n_estimators = 100), param_grid=param_grid)
model_RFC.fit(rvw_train,label_train)

KeyboardInterrupt: 

In [ ]:
predictionsRFC = model_RFC.predict(rvw_test)
print (classification_report(predictionsRFC, label_test))

In [ ]:
model_RFC.best_estimator_

In [ ]:
RFC = GridSearchCV(RandomForestClassifier(n_estimators = 100), param_grid=param_grid)

In [79]:
def classifier(X_train, y, clf, predict = 'evaluate', X_test = None):
    if predict == 'evaluate':
        name = clf.__class__.__name__
        rvw_train, rvw_test, label_train, label_test = \
        train_test_split(X_train, y, test_size=0.2) 
        clf.fit(rvw_train,label_train)
        predictionsCLF = clf.predict(rvw_test)
        print(name,"\n", classification_report(predictionsCLF, label_test))
    elif predict == 'submit':
        clf.fit(X,y)
        return clf.predict(X_test)

In [81]:
#Models based on best_estimator
model_MNB = MultinomialNB()

model_RFC = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy', max_depth=None, 
            max_features=10, max_leaf_nodes=None, min_impurity_split=1e-07, min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1, oob_score=False, random_state=None, verbose=0,
            warm_start=False)

model_SGD = SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1, eta0=0.0,
            fit_intercept=True, l1_ratio=0.15, learning_rate='optimal', loss='modified_huber', 
            n_iter=5, n_jobs=1, penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0, warm_start=False)

model_LR = LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, 
            max_iter=100, multi_class='ovr', n_jobs=1, penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False)

classifier(X_train, y_train, model_LR)
classifier(X_train, y_train, model_MNB)
classifier(X_train, y_train, model_RFC)
classifier(X_train, y_train, model_SGD)

LogisticRegression 
              precision    recall  f1-score   support

          0       0.86      0.88      0.87      2446
          1       0.89      0.86      0.87      2554

avg / total       0.87      0.87      0.87      5000

MultinomialNB 
              precision    recall  f1-score   support

          0       0.88      0.88      0.88      2529
          1       0.87      0.88      0.88      2471

avg / total       0.88      0.88      0.88      5000

RandomForestClassifier 
              precision    recall  f1-score   support

          0       0.84      0.87      0.85      2419
          1       0.87      0.84      0.86      2581

avg / total       0.85      0.85      0.85      5000

SGDClassifier 
              precision    recall  f1-score   support

          0       0.89      0.90      0.89      2459
          1       0.90      0.89      0.90      2541

avg / total       0.89      0.89      0.89      5000



In [113]:
from sklearn.ensemble import VotingClassifier

weights = {'weights': [[1, 1, 1, 1],[2, 2, 2, 1],[2, 2, 1, 2],[2, 1, 2, 2],[3, 3, 2, 1]]} 

model_Voting = GridSearchCV(VotingClassifier(estimators=[('SGD', model_SGD), ('LR', model_LR),
                                    ('MNB', model_MNB),('RFC', model_RFC)],
                                    voting='soft'), weights)

In [114]:
classifier(X_train, y_train, model_Voting)

GridSearchCV 
              precision    recall  f1-score   support

          0       0.90      0.91      0.90      2449
          1       0.91      0.90      0.90      2551

avg / total       0.90      0.90      0.90      5000



In [115]:
model_Voting.best_estimator_

VotingClassifier(estimators=[('SGD', SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
   ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         n_jobs=1, voting='soft', weights=[2, 1, 2, 2])

In [118]:
from sklearn.ensemble import VotingClassifier

weights = {'weights': [[3, 1, 2, 2],[2, 1, 3, 2],[2, 1, 2, 3],[2, 1, 2, 2]]} 

model_Voting = GridSearchCV(VotingClassifier(estimators=[('SGD', model_SGD), ('LR', model_LR),
                                    ('MNB', model_MNB),('RFC', model_RFC)],
                                    voting='soft'), weights)

In [119]:
classifier(X_train, y_train, model_Voting)
model_Voting.best_estimator_

GridSearchCV 
              precision    recall  f1-score   support

          0       0.90      0.90      0.90      2461
          1       0.90      0.90      0.90      2539

avg / total       0.90      0.90      0.90      5000



VotingClassifier(estimators=[('SGD', SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
   ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         n_jobs=1, voting='soft', weights=[2, 1, 2, 2])

In [ ]:
from IPython import embed
embed()

Python 3.6.0 |Anaconda custom (x86_64)| (default, Dec 23 2016, 13:19:00) 
Type "copyright", "credits" or "license" for more information.

IPython 5.1.0 -- An enhanced Interactive Python.
?         -> Introduction and overview of IPython's features.
%quickref -> Quick reference.
help      -> Python's own help system.
object?   -> Details about 'object', use 'object??' for extra details.

